In [95]:
 
from sys import version

print(version)

3.8.2 (default, Mar 11 2020, 00:28:52) 
[Clang 11.0.0 (clang-1100.0.33.17)]


In [65]:
import scipy
from app import AppContext
from core import LoadConfig
from app import Word
import numpy as np
from typing import *
from time import time

In [3]:
ctx = AppContext(LoadConfig("configs/main.yaml"))

[2020-04-11 15:45:41.938114] [  INFO   ] /34510/ LoadConfig core/config/__init__.py:13 LoadConfig: Load config source=configs/main.yaml


In [115]:
word_nsu_p = []
async for word in ctx.db.find(Word(word='нгу')):
    word_nsu_p.append(word.date)
word_nsu_ts = np.array(word_nsu_p)
word_nsu = np.unique(word_nsu_ts, return_counts=True)
word_nsu

(array([1182280301, 1223469024, 1234523182, ..., 1586421201, 1586425780,
        1586509840]),
 array([1, 1, 1, ..., 1, 1, 3]))

In [104]:
from scipy.signal import resample
from scipy.interpolate import interp1d

In [28]:
help(resample)

Help on function resample in module scipy.signal.signaltools:

resample(x, num, t=None, axis=0, window=None)
    Resample `x` to `num` samples using Fourier method along the given axis.
    
    The resampled signal starts at the same value as `x` but is sampled
    with a spacing of ``len(x) / num * (spacing of x)``.  Because a
    Fourier method is used, the signal is assumed to be periodic.
    
    Parameters
    ----------
    x : array_like
        The data to be resampled.
    num : int
        The number of samples in the resampled signal.
    t : array_like, optional
        If `t` is given, it is assumed to be the equally spaced sample
        positions associated with the signal data in `x`.
    axis : int, optional
        The axis of `x` that is resampled.  Default is 0.
    window : array_like, callable, string, float, or tuple, optional
        Specifies the window applied to the signal in the Fourier
        domain.  See below for details.
    
    Returns
    -------
 

[1182280301 1223469024 1234523182 ... 1586509840 1586509840 1586509840]
[        0  41188723  52242881 ... 404229539 404229539 404229539]
[ 1182280301  3096182963  5010085625  6923988287  8837890949 10751793611
 12665696273 14579598935 16493501597 18407404259 20321306921 22235209583
 24149112245 26063014907 27976917569 29890820232 31804722894 33718625556
 35632528218 37546430880 39460333542 41374236204 43288138866 45202041528
 47115944190 49029846852 50943749514 52857652176 54771554838 56685457500] [ 682.50310077  -76.13499213  160.06537893   96.36896008  219.13921838
  208.45643684  299.51211941  310.43457334  385.48620195  408.99960797
  473.67341463  506.07129127  562.88990237  602.43268418  652.58328095
  698.49969995  742.41731956  794.56671403  832.11070169  890.9281012
  921.32719797  987.99977213 1009.51442855 1086.56478031 1095.48855173
 1188.54285056 1175.86157089 1300.63008461 1234.93605121 1473.13099666]


(array([1913902662, 1913902662, 1913902662, 1913902662, 1913902662,
        1913902662, 1913902662, 1913902662, 1913902662, 1913902662,
        1913902662, 1913902662, 1913902662, 1913902662, 1913902663,
        1913902662, 1913902662, 1913902662, 1913902662, 1913902662,
        1913902662, 1913902662, 1913902662, 1913902662, 1913902662,
        1913902662, 1913902662, 1913902662, 1913902662]),
 array([-758.6380929 ,  236.20037106,  -63.69641885,  122.7702583 ,
         -10.68278155,   91.05568257,   10.92245394,   75.05162861,
          23.51340603,   64.67380666,   32.39787664,   56.8186111 ,
          39.5427818 ,   50.15059678,   45.916419  ,   43.91761961,
          52.14939447,   37.54398766,   58.81739951,   30.39909677,
          66.67257417,   21.51465642,   77.05035175,    8.92377143,
          93.05429883,  -12.68127968,  124.76851372,  -65.6940334 ,
         238.19494545]))

In [141]:
def slice(ts, vs):
    min_, p5, p95, max_ = ts.min(), \
    np.percentile(ts, 5), \
    np.percentile(ts, 95), \
    ts.max()
    
    _filter = (p5 < ts) & (ts < p95) & (ts > time() - 60 * 60 * 24 * 30 * 6)
    
    return ts[_filter], vs[_filter]

sliced_nsu = slice(*word_nsu)
print(sliced_nsu)

from scipy.interpolate import griddata

def res(t, v, count=None):
    count = count or (t.max() - t.min()) * 4 // np.percentile(np.diff(sliced_nsu[0]), 50) or 1
    count = int(count)
    print(t, count)
    xi = np.linspace(t.min(), t.max(), num=count, dtype=int)
    return xi, griddata(t, v, xi, method='linear')


resampled_nsu = res(*sliced_nsu)
resampled_nsu

(array([1571137057, 1571200489, 1571566310, 1571638504, 1571710104,
       1571762250, 1571843164, 1571915691, 1572361431, 1572404054,
       1572534748, 1572586389, 1572782392, 1572792796, 1572799025,
       1572810976, 1572847923, 1572874682, 1572948933, 1572983089,
       1573033323, 1573045403, 1573200727, 1573295821, 1573463030,
       1573563864, 1573616575, 1573787483, 1573822135, 1574152030,
       1574162083, 1574243360, 1574331968, 1574421321, 1574421467,
       1575278769, 1575340860, 1575652691, 1575806714, 1575890887,
       1575909346, 1575959753, 1575961005, 1576208345, 1576330907,
       1576511147, 1576673820, 1576744889, 1576920289, 1576998720,
       1577068115, 1577125426, 1577246221, 1577552910, 1577638319,
       1577681240, 1577744399, 1577766212, 1577804434, 1577810621,
       1578150742, 1578155160, 1578591732, 1578790994, 1578827865,
       1578928983, 1578999462, 1579074298, 1579238294, 1579859752,
       1579860731, 1579908581, 1580103898, 1580122446, 158026

(array([1571137057, 1571150966, 1571164875, 1571178784, 1571192693,
        1571206602, 1571220511, 1571234420, 1571248330, 1571262239,
        1571276148, 1571290057, 1571303966, 1571317875, 1571331784,
        1571345694, 1571359603, 1571373512, 1571387421, 1571401330,
        1571415239, 1571429148, 1571443058, 1571456967, 1571470876,
        1571484785, 1571498694, 1571512603, 1571526512, 1571540422,
        1571554331, 1571568240, 1571582149, 1571596058, 1571609967,
        1571623876, 1571637786, 1571651695, 1571665604, 1571679513,
        1571693422, 1571707331, 1571721240, 1571735150, 1571749059,
        1571762968, 1571776877, 1571790786, 1571804695, 1571818604,
        1571832514, 1571846423, 1571860332, 1571874241, 1571888150,
        1571902059, 1571915968, 1571929878, 1571943787, 1571957696,
        1571971605, 1571985514, 1571999423, 1572013332, 1572027242,
        1572041151, 1572055060, 1572068969, 1572082878, 1572096787,
        1572110696, 1572124606, 1572138515, 1572

In [64]:
print(word_nsu_ts)
word_nsu_ts.min(), \
    np.percentile(word_nsu_ts, 5), \
    np.percentile(word_nsu_ts, 95), \
    word_nsu_ts.max()

[1182280301 1223469024 1234523182 ... 1586509840 1586509840 1586509840]


(1182280301, 1366785953.9, 1584946730.5, 1586509840)

In [144]:
from plotly import graph_objects as go


def draw_word(fig, word_ts: Tuple[np.ndarray, np.ndarray],
            name, opacity=None, mode=None):
    x = word_ts[0]
    # corresponding y axis values
    y = word_ts[1]

    from datetime import datetime
    x_dt = [datetime.fromtimestamp(ts) for ts in x]

    fig.add_trace(go.Scatter(x=x_dt, y=y,
                             name=name, opacity=opacity,
                             mode=mode))
    
def integrate(word: Tuple[np.ndarray, np.ndarray]):
    return word[0], np.cumsum(word[1])

def diff(word: Tuple[np.ndarray, np.ndarray]):
    return (word[0][:-1] + word[0][1:]) / 2, np.diff(word[1])


def moving_average(word, periods=12):
    data_set = word[1]
    weights = np.ones(periods) / periods
    return word[0], np.convolve(data_set, weights, mode='valid')


fig = go.Figure()
    
# draw_word(fig, word_nsu, name='source', opacity=0.6, mode='markers')

# draw_word(fig, sliced_nsu, name='sliced')
prob_dist = diff(res(*integrate(sliced_nsu)))

# draw_word(fig, prob_dist, name='resampled')
draw_word(fig, moving_average(prob_dist), name='resampled mov avg')

# draw_word(fig, , name='int')
# draw_word(fig, diff(sliced_nsu), name='diff')

fig.show()

[1571137057 1571200489 1571566310 1571638504 1571710104 1571762250
 1571843164 1571915691 1572361431 1572404054 1572534748 1572586389
 1572782392 1572792796 1572799025 1572810976 1572847923 1572874682
 1572948933 1572983089 1573033323 1573045403 1573200727 1573295821
 1573463030 1573563864 1573616575 1573787483 1573822135 1574152030
 1574162083 1574243360 1574331968 1574421321 1574421467 1575278769
 1575340860 1575652691 1575806714 1575890887 1575909346 1575959753
 1575961005 1576208345 1576330907 1576511147 1576673820 1576744889
 1576920289 1576998720 1577068115 1577125426 1577246221 1577552910
 1577638319 1577681240 1577744399 1577766212 1577804434 1577810621
 1578150742 1578155160 1578591732 1578790994 1578827865 1578928983
 1578999462 1579074298 1579238294 1579859752 1579860731 1579908581
 1580103898 1580122446 1580268565 1580311482 1580312778 1580380293
 1580650675 1580734260 1580793628 1580801677 1580807877 1580960065
 1581057013 1581271337 1581306970 1581310770 1581319533 158133

In [125]:
np.percentile(np.diff(sliced_nsu[0]), 50)

55565.5